In [1]:
import pandas as pd
import plotly
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
import plotly.express as px
import numpy as np

In [2]:
data = pd.read_csv('../datasets/clean/datosArtsy.csv')

In [3]:
data.dropna(inplace=True)
data.shape

(6727, 14)

In [4]:
#Eliminar fracciones y castear a valores nnuméricos
data.height = data.height.str.split(' ')
data.length = data.length.str.split(' ')
data.height = data.height.apply(lambda x: x[0])
data.length = data.length.apply(lambda x: x[0])
data.height = data.height.astype(float)
data.length = data.length.astype(float)
data.date_text = data.date_text.astype(float)
data["sale_date"] = pd.to_datetime(data["sale_date"])
data["sale_date"] = data["sale_date"].astype(int)

In [5]:
data.head()

,title,height,length,art_type,medium_text,date_text,age,house,sale_date,price,author,performance,source,country
1,i) Untitled; ii) Untitled; iii) Untitled,33.0,24.8,Work on Paper,gouache and graphite on paper; watercolor and ...,1960.0,33.0,Christie's,-21555603000000,4000.0,agustin cardenas,0.33,artsy,cuba
3,Study for a sculpture,74.5,54.5,Work on Paper,Pastel,1970.0,43.0,Artcuria,-21556729000000,1810.0,agustin cardenas,1.40,artsy,cuba
5,Sans titre,49.0,63.0,Work on Paper,ink on paper,1975.0,48.0,Pias,-21557179000000,3655.0,agustin cardenas,0.35,artsy,cuba
7,Petite stèle,24.0,18.0,Sculpture,sculpted wood,1975.0,48.0,Ferri et Associe,-21557295000000,5120.0,agustin cardenas,0.37,artsy,cuba
12,Sans titre,58.0,34.0,Work on Paper,"charcoal, gouache and watercolor on paper",1969.0,42.0,Cornette de Saint Cy,-21557470000000,1883.0,agustin cardenas,0.67,artsy,cuba


In [6]:
#Codificacción de variables
def Encoder(df):
  columnsToEncode = list(df.select_dtypes(include=['category','object']))
  le = LabelEncoder()
  for feature in columnsToEncode:
      try:
          df[feature] = le.fit_transform(df[feature])
      except:
          print('Error encoding '+feature)
  return df

data_codificada = Encoder(data)

In [7]:
data_codificada.head()

,title,height,length,art_type,medium_text,date_text,age,house,sale_date,price,author,performance,source,country
1,4776,33.0,24.8,6,844,1960.0,33.0,41,-21555603000000,4000.0,0,0.33,0,6
3,4013,74.5,54.5,6,298,1970.0,43.0,10,-21556729000000,1810.0,0,1.40,0,6
5,3793,49.0,63.0,6,1022,1975.0,48.0,146,-21557179000000,3655.0,0,0.35,0,6
7,3363,24.0,18.0,3,1623,1975.0,48.0,69,-21557295000000,5120.0,0,0.37,0,6
12,3793,58.0,34.0,6,567,1969.0,42.0,47,-21557470000000,1883.0,0,0.67,0,6


In [21]:
len(data_codificada.date_text.unique())

128

In [22]:
features = ['height', 'length', 'art_type', 'date_text',
       'age', 'sale_date', 'price', 'author','country']
# Separating out the features
x = data_codificada.loc[:, features].values
#Estandarizar los datos
x = StandardScaler().fit_transform(x)

In [23]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [24]:
fig = px.scatter(x=principalDf["principal component 1"], y=principalDf["principal component 2"])
fig.show()

In [25]:
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])

In [26]:
fig = px.scatter_3d(x=principalDf["principal component 1"], y=principalDf["principal component 2"],
                    z=principalDf['principal component 3'])
fig.show()

In [13]:
#PCA con nuevos atributos y eliminando algunos de los anteriores
data_codificada = data_codificada.assign(precio_cm2 = lambda x: (x['price']/(x['height']*x['length'])))
data_codificada=data_codificada[~data_codificada.isin([np.nan, np.inf, -np.inf]).any(1)]

In [14]:
features = ['art_type', 'medium_text', 'sale_date', 'author','precio_cm2']
# Separating out the features
x = data_codificada.loc[:, features].values
#Estandarizar los datos
x = StandardScaler().fit_transform(x)

In [15]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [16]:
fig = px.scatter(x=principalDf["principal component 1"], y=principalDf["principal component 2"])
fig.show()